<a href="https://colab.research.google.com/github/chihchao/application-of-programming/blob/main/web_system_cluster_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
url = 'https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv'
df = pd.read_csv(url)

In [6]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[['女','男']])
df[['性別']] = encoder.fit_transform(df[['性別']])
df

,性別,年齡,收入（千）,消費指數（1~100）
0,0.0,74,38,81
1,0.0,51,71,91
2,0.0,30,65,10
3,0.0,88,49,17
4,0.0,55,48,70
...,...,...,...,...
195,1.0,86,84,82
196,1.0,59,52,30
197,0.0,63,29,61
198,1.0,67,80,9


In [7]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)
df['cluster'] = kmeans.fit_predict(df)
df

,性別,年齡,收入（千）,消費指數（1~100）,cluster
0,0.0,74,38,81,2
1,0.0,51,71,91,2
2,0.0,30,65,10,1
3,0.0,88,49,17,1
4,0.0,55,48,70,2
...,...,...,...,...,...
195,1.0,86,84,82,2
196,1.0,59,52,30,1
197,0.0,63,29,61,2
198,1.0,67,80,9,1


In [11]:
df.groupby('cluster').mean()

,性別,年齡,收入（千）,消費指數（1~100）
cluster,,,,
0,0.584906,28.962264,55.226415,72.018868
1,0.560976,56.414634,60.487805,21.439024
2,0.569231,67.707692,59.661538,75.953846


In [8]:
import joblib

# 保存模型與編碼器
joblib.dump(kmeans, 'kmeans_model.pkl')
joblib.dump(encoder, 'encoder.pkl')


['encoder.pkl']

In [9]:
!pip install -q gradio scikit-learn joblib pandas numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.7 MB/s eta 0:00:00


In [12]:
import gradio as gr
import joblib
import numpy as np

# 載入模型與編碼器
kmeans = joblib.load('kmeans_model.pkl')
encoder = joblib.load('encoder.pkl')

# 定義預測函數
def predict_cluster(gender, age, income, score):
    # 編碼性別
    gender_encoded = encoder.transform([[gender]])[0][0]

    # 建立特徵向量
    features = np.array([[gender_encoded, age, income, score]])

    # 預測分群
    cluster = kmeans.predict(features)[0]
    return f"此使用者屬於第 {cluster} 群"

markdown_table = """
| 群 | 性別 | 年齡 | 收入（千） | 消費指數（1~100） |
|---------|------|------|------------|--------------------|
| 0       | 0.6  | 29   | 55         | 72                 |
| 1       | 0.6  | 56   | 60         | 21                 |
| 2       | 0.6  | 68   | 60         | 76                 |
"""

# 建立介面
interface = gr.Interface(
    fn=predict_cluster,
    inputs=[
        gr.Radio(["女", "男"], label="性別"),
        gr.Slider(10, 70, label="年齡"),
        gr.Slider(0, 150, label="收入（千）"),
        gr.Slider(0, 100, label="消費分數")
    ],
    outputs=[gr.Markdown(markdown_table),"text"],
    title="客戶分群預測器"
)

# 啟動介面
interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ee393b3de4963b1b8f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
